# Getting Started

In [1]:
import math
import time
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import paste as pst
import ot
import plotly.express as px
import plotly.io as pio
from re import S
import scanpy as sc
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from scipy.spatial import distance_matrix
import random
import sklearn.metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import adjusted_rand_score
import scanpy as sc
import scipy

2024-01-10 20:16:36.473584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/lcl/mambaforge/envs/eggplant/lib/python3.8/site-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


## Read data and create AnnData object

In [2]:
def load_slices_h5(data_dir, file_names):
    slices = []
    for file_name in file_names:
        slice_i = sc.read_h5ad(data_dir + file_name)
        if scipy.sparse.issparse(slice_i.X):
            slice_i.X = slice_i.X.toarray()

        n_counts = slice_i.obs['n_counts']
        layer_guess_reordered = slice_i.obs['layer_guess_reordered']
        slice_i.obsm['spatial'] = slice_i.obs[['imagerow', 'imagecol']].to_numpy()
        slice_i.obs = pd.DataFrame({'n_counts': n_counts, 'layer_guess_reordered': layer_guess_reordered})
        slice_i.var = pd.DataFrame({'n_counts': slice_i.var['n_counts']})
        sc.pp.filter_genes(slice_i, min_counts = 15)
        sc.pp.filter_cells(slice_i, min_counts = 100)
        slices.append(slice_i)
    return slices

In [3]:
class visualize:
    def __init__(self,slices,slice_colors):
        self.slices = slices
        self.slice_colors = slice_colors
        self.center_color = 'orange'
    def spatial_coordinates(self):
        num_slices = len(self.slices)
        num_rows = int(num_slices**0.5)
        num_cols = int(num_slices / num_rows) + (num_slices % num_rows > 0)
        
        fig, axs = plt.subplots(num_rows, num_cols, figsize=(7, 7)) 
        axs = axs.flatten() 
        
        for i, (slice_i, color) in enumerate(zip(self.slices, self.slice_colors)):
            pst.plot_slice(slice_i, color, ax=axs[i])
        
        # If there are any empty subplots, hide them
        for j in range(i + 1, len(axs)):
            axs[j].axis('off')  
        plt.tight_layout()
        plt.show()  
    def spatial_counts(self):
        for slice_i in slices:    
            sc.pl.spatial(slice_i, color="n_counts", spot_size=1)

    def new_slice(self,new_slices):
        for i in range(len(new_slices)):
            pst.plot_slice(new_slices[i],self.slice_colors[i],s=400)
        plt.legend(handles=[mpatches.Patch(color=self.slice_colors[0], label='1'),mpatches.Patch(color=self.slice_colors[1], label='2'),mpatches.Patch(color=self.slice_colors[2], label='3'),mpatches.Patch(color=slice_colors[3], label='4')])
        plt.gca().invert_yaxis()
        plt.axis('off')
        plt.show()

    def plot_pairwise_layers(self,new_slices):
        num_slices = len(new_slices)
        num_plots = num_slices - 1

        num_rows = num_plots // 2 + num_plots % 2
        num_cols = 2
        fig, axs = plt.subplots(num_rows, num_cols, figsize=(7, 7))
        axs = axs.flatten() if num_plots > 1 else [axs]

        for i in range(num_plots):
            pst.plot_slice(new_slices[i], self.slice_colors[i % len(self.slice_colors)], ax=axs[i])
            pst.plot_slice(new_slices[i + 1], self.slice_colors[(i + 1) % len(self.slice_colors)], ax=axs[i])

        for j in range(num_plots, len(axs)):
            fig.delaxes(axs[j])

        plt.tight_layout()
        plt.show()
        
    def new_slices_3d(self,new_slices):
        pio.renderers.default='notebook'
        z_scale = 2

        values = []
        for i,L in enumerate(new_slices):
            for x,y in L.obsm['spatial']:
                values.append([x, y, i*z_scale, str(i)])
        df = pd.DataFrame(values, columns=['x','y','z','slice'])
        fig = px.scatter_3d(df, x='x', y='y', z='z',
                    color='slice',color_discrete_sequence=self.slice_colors)
        fig.update_layout(scene_aspectmode='data')
        fig.show()

    def center_align_stack(self,center,new_slices):
        plt.figure(figsize=(7,7))
        pst.plot_slice(center,self.center_color,s=400)
        for i in range(len(new_slices)):
            pst.plot_slice(new_slices[i],self.slice_colors[i],s=400)

        legend_handles = [mpatches.Patch(color=color, label=str(i + 1)) for i, color in enumerate(self.slice_colors)]
        plt.legend(handles=legend_handles)
        plt.gca().invert_yaxis()
        plt.axis('off')
        plt.show()
    
    def center_align_center(self,center,new_slices):
        num_slices = len(new_slices)
        num_plots = num_slices

        num_rows = num_plots // 2 + num_plots % 2
        num_cols = 2

        fig, axs = plt.subplots(num_rows, num_cols, figsize=(7, 7))
        axs = axs.flatten() if num_plots > 1 else [axs]

        for i in range(num_slices):
            pst.plot_slice(center, self.center_color, ax=axs[i])
            pst.plot_slice(new_slices[i], self.slice_colors[i], ax=axs[i])

        for j in range(num_slices, len(axs)):
            fig.delaxes(axs[j])

        plt.tight_layout()
        plt.show()

In [4]:
def align_slices(slices, use_gpu=True):
    alignments = {}
    backend = ot.backend.TorchBackend()
    start = time.time()

    for i in range(len(slices) - 1):
        pi = pst.pairwise_align(slices[i], slices[i + 1], backend=backend, use_gpu=use_gpu)
        alignments[f'pi{i+1}{i+2}'] = pi

    print('Runtime: ' + str(time.time() - start))
    return alignments

In [5]:
def generate_leiden_cluster(slices, n_top_genes=2000, n_pcs=50):
    processed_slices = [slice.copy() for slice in slices]

    for slice in processed_slices:
        sc.pp.normalize_total(slice, target_sum=1e4)
        sc.pp.log1p(slice)
        sc.pp.highly_variable_genes(slice, n_top_genes=n_top_genes)
        sc.pp.pca(slice, n_comps=n_pcs)
        sc.pp.neighbors(slice)
        sc.tl.leiden(slice)
        slice.obs['layer_guess_reordered'] = slice.obs['leiden']

    return processed_slices

In [6]:
def add_cluster_labels_to_original_slices(original_slices, processed_slices):
    for original_slice, processed_slice in zip(original_slices, processed_slices):
        original_slice.obs['layer_guess_reordered'] = processed_slice.obs['layer_guess_reordered']
    return original_slices

In [7]:
def aligned_slices_with_label(slices):
    # processed_slices = generate_leiden_cluster(slices)
    # original_slices_with_label = add_cluster_labels_to_original_slices(slices,processed_slices)
    alignments = align_slices(slices)
    pis = [alignments[f'pi{i}{i+1}'] for i in range(1, len(slices))]
    new_slices = pst.stack_slices_pairwise(slices, pis)
    return  pis,new_slices

# Metric 1 PAA

In [8]:
def create_binary_matrix(slice, n_categories):
    binary_matrix = np.zeros((slice.n_obs, n_categories))
    for idx, cat in enumerate(slice.obs['layer_guess_reordered']):
        binary_matrix[idx, cat - 1] = 1  
    return binary_matrix

In [9]:
data_dir = "./../../example_dataset/spatial/"
file_names = ["151507_preprocessed.h5", "151508_preprocessed.h5"]
slices = load_slices_h5(data_dir, file_names)
mapping_dict = {'Layer1':1, 'Layer2':2, 'Layer3':3, 'Layer4':4, 'Layer5':5, 'Layer6':6, 'WM':7}
backend = ot.backend.TorchBackend()
use_gpu=True
for slice_i in slices:
    slice_i.obs['layer_guess_reordered'] = slice_i.obs['layer_guess_reordered'].map(mapping_dict)
    
total_accuracy = 0
n_categories = len(mapping_dict)  


In [10]:
for i in range(len(slices)):
    for j in range(len(slices)):
        if i != j:
            binary_matrix_i = create_binary_matrix(slices[i], n_categories)
            binary_matrix_j = create_binary_matrix(slices[j], n_categories)
            pi = pst.pairwise_align(slices[i], slices[j], backend=backend, use_gpu=use_gpu)
            matched_pairs = np.dot(binary_matrix_i, binary_matrix_j.T)
            total_accuracy += np.sum(pi * matched_pairs)

ave_accuracy = total_accuracy / (len(slices) * (len(slices) - 1))

print(ave_accuracy)

gpu is available, using gpu.


/home/lcl/mambaforge/envs/eggplant/lib/python3.8/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


gpu is available, using gpu.
0.27976604027372787


# Metrics2 Spatial Coherence

In [24]:
def create_graph(adata, degree = 4):
        """
        Converts spatial coordinates into graph using networkx library.
        
        param: adata - ST Slice 
        param: degree - number of edges per vertex

        return: 1) G - networkx graph
                2) node_dict - dictionary mapping nodes to spots
        """
        D = distance_matrix(adata.obsm['spatial'], adata.obsm['spatial'])
        # Get column indexes of the degree+1 lowest values per row
        idx = np.argsort(D, 1)[:, 0:degree+1]
        # Remove first column since it results in self loops
        idx = idx[:, 1:]

        G = nx.Graph()
        for r in range(len(idx)):
            for c in idx[r]:
                G.add_edge(r, c)

        node_dict = dict(zip(range(adata.shape[0]), adata.obs.index))
        return G, node_dict
    
def generate_graph_from_labels(adata, labels_dict):
    """
    Creates and returns the graph and dictionary {node: cluster_label} for specified layer
    """
    
    g, node_to_spot = create_graph(adata)
    spot_to_cluster = labels_dict

    # remove any nodes that are not mapped to a cluster
    removed_nodes = []
    for node in node_to_spot.keys():
        if (node_to_spot[node] not in spot_to_cluster.keys()):
            removed_nodes.append(node)

    for node in removed_nodes:
        del node_to_spot[node]
        g.remove_node(node)
        
    labels = dict(zip(g.nodes(), [spot_to_cluster[node_to_spot[node]] for node in g.nodes()]))
    return g, labels


def spatial_entropy(g, labels):
    """
    Calculates spatial entropy of graph  
    """
    # construct contiguity matrix C which counts pairs of cluster edges
    cluster_names = np.unique(list(labels.values()))
    C = pd.DataFrame(0,index=cluster_names, columns=cluster_names)

    for e in g.edges():
        C[labels[e[0]]][labels[e[1]]] += 1

    # calculate entropy from C
    C_sum = C.values.sum()
    H = 0
    for i in range(len(cluster_names)):
        for j in range(i, len(cluster_names)):
            if (i == j):
                z = C[cluster_names[i]][cluster_names[j]]
            else:
                z = C[cluster_names[i]][cluster_names[j]] + C[cluster_names[j]][cluster_names[i]]
            if z != 0:
                H += -(z/C_sum)*math.log(z/C_sum)
    return H



def spatial_coherence_score(graph, labels):
    g, l = graph, labels
    true_entropy = spatial_entropy(g, l)
    entropies = []
    for i in range(100):
        new_l = list(l.values())
        random.shuffle(new_l)
        labels = dict(zip(l.keys(), new_l))
        entropies.append(spatial_entropy(g, labels))
        
    return abs((true_entropy - np.mean(entropies))/np.std(entropies))

In [25]:
def average_spatial_coherence_score(aligned_slices):
    total_score = 0
    num_slices = len(aligned_slices)
    for aligned_slice in aligned_slices:
        labels_dict = dict(zip(aligned_slice.obs.index, aligned_slice.obs['layer_guess_reordered']))
        g, labels = generate_graph_from_labels(aligned_slice, labels_dict)

        score = spatial_coherence_score(g, labels)
        total_score += score

    average_spatial_coherence_score = total_score / num_slices
    print("Average Spatial Coherence Score:", average_spatial_coherence_score)

In [26]:
data_dir = "./../../example_dataset/spatial/"
file_names = ["151507_preprocessed.h5", "151508_preprocessed.h5"]
slices = load_slices_h5(data_dir, file_names)
mapping_dict = {'Layer1':1, 'Layer2':2, 'Layer3':3, 'Layer4':4, 'Layer5':5, 'Layer6':6, 'WM':7}
for slice_i in slices:
    slice_i.obs['layer_guess_reordered'] = slice_i.obs['layer_guess_reordered'].map(mapping_dict)


_, aligned_slices = aligned_slices_with_label(slices)
average_spatial_coherence_score(aligned_slices)

gpu is available, using gpu.
gpu is available, using gpu.
gpu is available, using gpu.
Runtime: 9.939272403717041
Average Spatial Coherence Score: 266.42704117007884


# Metrics 3  LTARI

In [29]:
def compute_average_ltari(slices, k=1):
    average_ltari_all_slices = []
    for ref_index in range(len(slices)):
        reference_slice = slices[ref_index]
        ref_coords = reference_slice.obsm['spatial']
        nn_model = NearestNeighbors(n_neighbors=k).fit(ref_coords)
        ltari_values = []
        for query_index in range(len(slices)):
            if query_index != ref_index:
                query_slice = slices[query_index]
                query_coords = query_slice.obsm['spatial']
                _, nearest_indices = nn_model.kneighbors(query_coords)
                if k == 1:
                    transferred_labels = reference_slice.obs['layer_guess_reordered'].iloc[nearest_indices.flatten()].values
                else:
                    transferred_labels = np.array([reference_slice.obs['layer_guess_reordered'].iloc[indices].mode()[0] for indices in nearest_indices])
                ari = adjusted_rand_score(query_slice.obs['layer_guess_reordered'], transferred_labels)
                ltari_values.append(ari)
        average_ltari_all_slices.append(np.mean(ltari_values))
    final_average_ltari = np.mean(average_ltari_all_slices)
    return final_average_ltari


In [30]:
data_dir = "./../../example_dataset/spatial/"
file_names = ["151507_preprocessed.h5", "151508_preprocessed.h5"]
slices = load_slices_h5(data_dir, file_names)
mapping_dict = {'Layer1':1, 'Layer2':2, 'Layer3':3, 'Layer4':4, 'Layer5':5, 'Layer6':6, 'WM':7}
backend = ot.backend.TorchBackend()
use_gpu=True
for slice_i in slices:
    slice_i.obs['layer_guess_reordered'] = slice_i.obs['layer_guess_reordered'].map(mapping_dict)

_, aligned_slices = aligned_slices_with_label(slices)
compute_average_ltari(aligned_slices)

gpu is available, using gpu.


/home/lcl/mambaforge/envs/eggplant/lib/python3.8/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


gpu is available, using gpu.
gpu is available, using gpu.
Runtime: 100.76190519332886


0.6299838940364161